In [ ]:
import numpy as np
from dreye.algebra.filtering import Filter1D
import matplotlib.pyplot as plt

In [ ]:
arr = np.array([100]*10+[0] + [100] * 10 + [0] + [100]*10)

In [ ]:
window = 9

In [ ]:
lfilter = Filter1D('boxcar', window)

In [ ]:
start_idx, end_idx = int(np.floor((window-1)/2)), int(np.ceil((window-1)/2))

In [ ]:
start_idx, end_idx

In [ ]:
new_arr = np.hstack([
    np.array([100] * (start_idx)),
    arr, 
    np.array([100] * (end_idx)),
])

In [ ]:
arr_filt = lfilter(new_arr, axis=-1, mode='valid')
arr_filt.shape, arr.shape

In [ ]:
from dreye.core import Signal

In [ ]:
t = np.arange(-5, 5, 0.01)
a = np.array([3, 4, 5])
phase = np.array([0.1, -0.5, 1]) * np.pi

x = a[None, :] * np.sin(t[:, None] + phase[None, :])

x_noise = x + np.random.normal(0, 0.7, size=x.shape)

In [ ]:
signal = Signal(
    x_noise, 
    units='mV', 
    domain=t, 
    domain_units='s', 
    interpolator_kwargs={
        'bounds_error':False, 
        'fill_value':'extrapolate'
    }
)

In [ ]:
signal.domain.interval

In [ ]:
filtered_signal = signal.window_filter(1, 'hann')

In [ ]:
plt.plot(signal.domain, signal.values)
plt.plot(filtered_signal.domain, filtered_signal.values)
# TODO smooth interpolation

In [ ]:
from dreye.core import LogPhotoreceptor, \
    RelativeOpsinSensitivity, Spectrum, Domain
from scipy.stats import norm

In [ ]:
wl = np.arange(300, 600)
s = norm.pdf(wl, 450, 10)
s /= np.sum(s)
s_noise = s + np.random.normal(0, 0.001, size=s.shape)

In [ ]:
opsin_noise = RelativeOpsinSensitivity(
    s_noise, 
    domain=wl, 
    smoothing_method='savgol', 
    smoothing_window=15, 
    smoothing_args={'polyorder':2}
)

opsin = RelativeOpsinSensitivity(
    s, 
    domain=wl, 
    smoothing_method='savgol', 
    smoothing_window=15, 
    smoothing_args={'polyorder':2}
)

In [ ]:
opsin_interp = opsin(
    Domain(np.arange(400, 550), units='nm')
)
plt.plot(opsin_interp.domain, opsin_interp)

In [ ]:
opsin_noise.smoothing_method

In [ ]:
opsin_smooth2 = opsin_noise.smooth

In [ ]:
opsin_smooth = opsin_noise.window_filter(
    15, 'savgol', polyorder=2
)

In [ ]:
type(opsin_smooth), type(opsin_smooth2)

In [ ]:
from scipy.signal import savgol_filter

In [ ]:
ssmooth = savgol_filter(s_noise, 15, 2)

In [ ]:
plt.plot(opsin.wavelengths, opsin)
plt.plot(opsin_noise.wavelengths, opsin_noise)
plt.plot(opsin_smooth.wavelengths, opsin_smooth)

In [ ]:
pr = LogPhotoreceptor(opsin_interp)
pr2 = LogPhotoreceptor(opsin_smooth)

In [ ]:
ill = norm.pdf(wl, 400, 80)
bg = np.ones(wl.shape)

In [ ]:
illuminant = Spectrum(
    ill, 
    domain=wl
)
background = Spectrum(
    bg, 
    domain=wl
)

In [ ]:
np.log(np.sum(ill*s)/np.sum(bg*s))

In [ ]:
pr.excitation(
    illuminant, 
    background=background, 
    units=False
)

In [ ]:
pr2.excitation(
    illuminant, 
    background=background, 
    units=False
)

In [ ]:
np.log(np.sum(ill*s_noise)/np.sum(bg*s_noise))

In [ ]:
type(illuminant)

In [ ]:
illuminant.normalized_spectrum

In [ ]:
illph = illuminant.photonflux